In [ ]:
%pip install -qU langchain-openai langchain langgraph langchain_community langgraph-supervisor

In [ ]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage,
)
from langgraph.prebuilt import create_react_agent
import requests
key = userdata.get('OPENAI_API_KEY')

In [ ]:
model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

# Tool 1

In [ ]:
from langchain_core.tools import tool
import time

@tool
def guided_meditation_tool(style: str = "mindfulness", duration: int = 10) -> str:
    """
    Provide a step-by-step guided meditation process with breaks.

    Args:
        style (str): The type of meditation (e.g., 'mindfulness', 'breathing', 'body_scan').
        duration (int): Total duration in minutes (suggested: 5–20 mins).

    Returns:
        str: Step-by-step meditation guide with optional pause indicators.
    """
    steps = {
        "mindfulness": [
            "🧘 Step 1: Find a comfortable seated position. Close your eyes gently.",
            "🌬️ Step 2: Bring attention to your breath. Inhale deeply through your nose, and exhale through your mouth.",
            "🔄 Step 3: Focus on the rising and falling of your breath. If your mind wanders, gently bring it back.",
            "💭 Step 4: Acknowledge any thoughts or feelings without judgment. Let them pass like clouds.",
            "🧠 Step 5: Expand awareness to sounds around you, sensations in the body, and your emotional state.",
            "💖 Step 6: End by bringing attention to your heart. Think of one thing you're grateful for.",
            "🔚 Step 7: Slowly open your eyes and return to the space around you."
        ],
        "breathing": [
            "🪑 Step 1: Sit or lie down comfortably.",
            "🫁 Step 2: Breathe in for 4 seconds, hold for 4 seconds, exhale for 6 seconds.",
            "🔁 Step 3: Repeat this cycle 10 times.",
            "🎯 Step 4: Focus on the sensation of air passing through your nose and lungs.",
            "🔄 Step 5: If distracted, calmly return to your breath.",
            "🧘 Step 6: After a few minutes, let your breath return to normal.",
            "🔚 Step 7: Take one deep final breath and slowly open your eyes."
        ],
        "body_scan": [
            "🛏️ Step 1: Lie down comfortably and close your eyes.",
            "🧠 Step 2: Bring awareness to your head, relaxing your forehead, eyes, jaw.",
            "💪 Step 3: Move attention down to your shoulders, arms, and hands.",
            "🫁 Step 4: Continue to chest, belly, and hips—breathe into each area.",
            "🦵 Step 5: Bring awareness to thighs, knees, calves, and feet.",
            "🔁 Step 6: Do a full body scan from head to toe once more.",
            "🔚 Step 7: When ready, gently wiggle your fingers and toes and open your eyes."
        ]
    }

    if style not in steps:
        return "❌ Unknown meditation style. Choose from: 'mindfulness', 'breathing', or 'body_scan'."

    guide = steps[style]
    segment_duration = duration // len(guide)

    result = f"🧘 Guided {style.replace('_', ' ').title()} Meditation ({duration} mins):\n"
    for i, step in enumerate(guide, start=1):
        result += f"\n{i}. {step}\n"
        result += f"⏸️ (Pause for ~{segment_duration} minutes)\n"

    return result.strip()


In [ ]:
meditation_assistant = create_react_agent(
    model=model,
    tools=[guided_meditation_tool],
    prompt="You are a helpful Meditation expert. Your job is to help users do meditation based on their queries.",
    name="meditation_assistant"
)

In [ ]:
response = meditation_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="upper body")
            ]
    }
    )
response["messages"]

[HumanMessage(content='upper body', additional_kwargs={}, response_metadata={}, id='e4ca9db2-5da2-4bc4-8031-04a4fa719da0'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_OH7dXBCXtoqJqQb9Q4nQ25L9', 'function': {'arguments': '{"style":"body_scan","duration":10}', 'name': 'guided_meditation_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 151, 'total_tokens': 174, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BgwnZz9cXb0Op4TUChbcxBKd4mk7E', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='meditation_assistant', id='run--fd8c5e4b-ae3a-496e-a3ab-7468df61e980-0', tool_calls=[{'name': 'guided_meditation_tool', 

In [ ]:
response["messages"][-1].content

"Here's a guided body scan meditation focusing on the upper body that lasts for 10 minutes:\n\n🧘 **Guided Body Scan Meditation (10 mins)**:\n\n1. **🛏️ Step 1:** Lie down comfortably and close your eyes.  \n   ⏸️ (Pause for ~1 minute)\n\n2. **🧠 Step 2:** Bring awareness to your head. Relax your forehead, eyes, and jaw.  \n   ⏸️ (Pause for ~1 minute)\n\n3. **💪 Step 3:** Move your attention down to your shoulders, arms, and hands. Notice any tension and let it go.  \n   ⏸️ (Pause for ~1 minute)\n\n4. **🫁 Step 4:** Shift your focus to your chest and belly. Breathe into each area, feeling it expand and contract.  \n   ⏸️ (Pause for ~1 minute)\n\n5. **🦵 Step 5:** Bring awareness to the upper part of your thighs and pelvic area. Release any tightness you may feel.  \n   ⏸️ (Pause for ~1 minute)\n\n6. **🔁 Step 6:** Do a full body scan from head to toe once more, noticing how your body feels after the meditation.  \n   ⏸️ (Pause for ~1 minute)\n\n7. **🔚 Step 7:** When ready, gently wiggle your 

# Tool 2

In [ ]:
from langchain_core.tools import tool
import random

@tool
def guided_meditation_tool2(style: str = "mindfulness", duration: int = 10, intensity: str = "relaxing") -> str:
    """
    Generate a dynamic, step-by-step guided meditation session with varied instructions and timed breaks.

    Args:
        style (str): Type of meditation ('mindfulness', 'breathing', or 'body_scan').
        duration (int): Total duration in minutes.
        intensity (str): Type of mood or tone ('relaxing', 'energizing').

    Returns:
        str: A multi-step, varied meditation script.
    """
    # Define intensity tone modifiers
    tone_prefix = {
        "relaxing": "Gently",
        "energizing": "With alert awareness"
    }

    # Define steps by meditation style
    instructions = {
        "mindfulness": [
            "observe your thoughts as they arise.",
            "bring awareness to your breath without trying to control it.",
            "notice the sensations in your body.",
            "focus on the feeling of the air against your skin.",
            "be fully present with each sound you hear.",
            "watch your thoughts pass like clouds in the sky.",
            "feel grounded by the contact between your body and the seat."
        ],
        "breathing": [
            "breathe in deeply through your nose, out through your mouth.",
            "use box breathing: inhale, hold, exhale, hold — all for 4 counts.",
            "let your breath become slower and deeper.",
            "place one hand on your chest, the other on your belly, and feel your breath.",
            "count each breath cycle: in and out equals one.",
            "pause after each inhale and exhale, feeling stillness.",
            "anchor your awareness in the rhythm of your breath."
        ],
        "body_scan": [
            "relax your forehead and soften your jaw.",
            "release tension from your shoulders.",
            "feel your chest rise and fall naturally.",
            "observe any sensations in your abdomen.",
            "notice your legs and let them become heavy.",
            "feel into your toes and feet — relax completely.",
            "move attention slowly from head to toe."
        ]
    }

    if style not in instructions:
        return "❌ Unknown meditation style. Choose from: 'mindfulness', 'breathing', or 'body_scan'."

    if intensity not in tone_prefix:
        return "❌ Unknown intensity. Choose from: 'relaxing', 'energizing'."

    steps = random.sample(instructions[style], k=min(len(instructions[style]), duration))
    pause_time = duration // len(steps)

    # Tone-based intro
    response = f"🧘 {style.title()} Meditation ({intensity.title()}) for {duration} minutes:\n"

    for i, step in enumerate(steps, 1):
        prefix = tone_prefix[intensity]
        response += f"\n{i}. {prefix}, {step}\n⏸️ Pause for ~{pause_time} minutes...\n"

    # Dynamic closings
    closing_relaxing = [
        "Gently bring your awareness back to the present.",
        "Wiggle your fingers and toes softly.",
        "Open your eyes when you're ready.",
        "Notice how your body feels now compared to when you started."
    ]
    closing_energizing = [
        "Take a deep breath and feel refreshed.",
        "Open your eyes with energy and clarity.",
        "Stretch your arms and prepare to move forward with focus.",
        "Smile and carry this awareness into the rest of your day."
    ]

    closing = random.choice(closing_relaxing if intensity == "relaxing" else closing_energizing)
    response += f"\n🔚 Final Step: {closing}"

    return response.strip()


In [ ]:
meditation_assistant2 = create_react_agent(
    model=model,
    tools=[guided_meditation_tool2],
    prompt="You are a helpful Meditation expert. Your job is to help users do meditation based on their queries.",
    name="meditation_assistant2"
)

In [ ]:
response2 = meditation_assistant2.invoke(
    {
        "messages": [
            HumanMessage(content="mindful meditation that is relaxing for 10 minutes.")
            ]
    }
    )
response2["messages"]

[HumanMessage(content='mindful meditation that is relaxing for 10 minutes.', additional_kwargs={}, response_metadata={}, id='ac51a73a-982c-44c4-93c2-d00bcf5c1583'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_JDCwTcgakXlt1Dcgr3raq6qO', 'function': {'arguments': '{"style":"mindfulness","duration":10,"intensity":"relaxing"}', 'name': 'guided_meditation_tool2'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 183, 'total_tokens': 214, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BgwuRWZftHcJYXGjs0H1kwEY26d8G', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='meditation_assistant2', id='run--46ee4703-0a8a-406c-b

In [ ]:
response2["messages"][-1].content

"Here's a 10-minute guided mindfulness meditation that is designed to be relaxing:\n\n### Mindfulness Meditation (Relaxing) for 10 Minutes\n\n1. **Focus on the Air**: Gently, focus on the feeling of the air against your skin.\n   ⏸️ Pause for ~1 minute...\n\n2. **Grounding**: Gently, feel grounded by the contact between your body and the seat.\n   ⏸️ Pause for ~1 minute...\n\n3. **Listening**: Gently, be fully present with each sound you hear.\n   ⏸️ Pause for ~1 minute...\n\n4. **Breath Awareness**: Gently, bring awareness to your breath without trying to control it.\n   ⏸️ Pause for ~1 minute...\n\n5. **Thought Observation**: Gently, observe your thoughts as they arise.\n   ⏸️ Pause for ~1 minute...\n\n6. **Clouds in the Sky**: Gently, watch your thoughts pass like clouds in the sky.\n   ⏸️ Pause for ~1 minute...\n\n7. **Sensations**: Gently, notice the sensations in your body.\n   ⏸️ Pause for ~1 minute...\n\n🔚 **Final Step**: Open your eyes when you're ready.\n\nTake your time with

# Class Local

# Meditation Tool. PY

In [ ]:
from langchain_core.tools import tool
import random

@tool
def guided_meditation_tool2(style: str = "mindfulness", duration: int = 10, intensity: str = "relaxing") -> str:
    """
    Generate a dynamic, step-by-step guided meditation session with varied instructions and timed breaks.

    Args:
        style (str): Type of meditation ('mindfulness', 'breathing', or 'body_scan').
        duration (int): Total duration in minutes.
        intensity (str): Type of mood or tone ('relaxing', 'energizing').

    Returns:
        str: A multi-step, varied meditation script.
    """
    # Define intensity tone modifiers
    tone_prefix = {
        "relaxing": "Gently",
        "energizing": "With alert awareness"
    }

    # Define steps by meditation style
    instructions = {
        "mindfulness": [
            "observe your thoughts as they arise.",
            "bring awareness to your breath without trying to control it.",
            "notice the sensations in your body.",
            "focus on the feeling of the air against your skin.",
            "be fully present with each sound you hear.",
            "watch your thoughts pass like clouds in the sky.",
            "feel grounded by the contact between your body and the seat."
        ],
        "breathing": [
            "breathe in deeply through your nose, out through your mouth.",
            "use box breathing: inhale, hold, exhale, hold — all for 4 counts.",
            "let your breath become slower and deeper.",
            "place one hand on your chest, the other on your belly, and feel your breath.",
            "count each breath cycle: in and out equals one.",
            "pause after each inhale and exhale, feeling stillness.",
            "anchor your awareness in the rhythm of your breath."
        ],
        "body_scan": [
            "relax your forehead and soften your jaw.",
            "release tension from your shoulders.",
            "feel your chest rise and fall naturally.",
            "observe any sensations in your abdomen.",
            "notice your legs and let them become heavy.",
            "feel into your toes and feet — relax completely.",
            "move attention slowly from head to toe."
        ]
    }

    if style not in instructions:
        return "❌ Unknown meditation style. Choose from: 'mindfulness', 'breathing', or 'body_scan'."

    if intensity not in tone_prefix:
        return "❌ Unknown intensity. Choose from: 'relaxing', 'energizing'."

    steps = random.sample(instructions[style], k=min(len(instructions[style]), duration))
    pause_time = duration // len(steps)

    # Tone-based intro
    response = f"🧘 {style.title()} Meditation ({intensity.title()}) for {duration} minutes:\n"

    for i, step in enumerate(steps, 1):
        prefix = tone_prefix[intensity]
        response += f"\n{i}. {prefix}, {step}\n⏸️ Pause for ~{pause_time} minutes...\n"

    # Dynamic closings
    closing_relaxing = [
        "Gently bring your awareness back to the present.",
        "Wiggle your fingers and toes softly.",
        "Open your eyes when you're ready.",
        "Notice how your body feels now compared to when you started."
    ]
    closing_energizing = [
        "Take a deep breath and feel refreshed.",
        "Open your eyes with energy and clarity.",
        "Stretch your arms and prepare to move forward with focus.",
        "Smile and carry this awareness into the rest of your day."
    ]

    closing = random.choice(closing_relaxing if intensity == "relaxing" else closing_energizing)
    response += f"\n🔚 Final Step: {closing}"

    return response.strip()


# Run_meditation agent.py

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from agents.meditation_agent import MeditationAgent
def main():
    agent = MeditationAgent()
    query = "mindful meditation that is relaxing for 10 minutes."
    result = agent.run(query)
    print(result)

if __name__ == "__main__":
    main()

# Meditation Agent py

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from tools.email_tools import guided_meditation_tool2
from config.setting import OPENAI_API_KEY

class MeditationAgent:
    def __init__(self):
        self.openai_api_key = OPENAI_API_KEY
        self.model = ChatOpenAI(api_key=self.openai_api_key, model="gpt-4o-mini", verbose=True)
        self.agent = create_react_agent(
            model=self.model,
            tools=[guided_meditation_tool2],
            prompt = (
              "You are a helpful Meditation expert. Your job is to help users do meditation "
              "based on their requests. Use the tool to generate steps for meditation."

            ),
            name="meditation_assistant"
        )

    def run(self, user_input):
        response = self.agent.invoke({
            "messages": [HumanMessage(content=user_input)]
        }, config={"recursion_limit": 50})
        return response["messages"][-1].content